# Overview 

For a given dataset, I would like to restore the relational database format. To reach this goal I created this notebook where I dump the data rows of a Data Frame into a SQL file made of INSERT statements.

This notebook is also designed to help me to learn more about the Pandas library and Python language. So the strategy used here was pretty simple. It was not intended to achieve the best solution.

The resulted sql files was loaded in a MySQL database. This is a resource for a workshop where I introduce SQL concepts to a Data Science students group and let they try their first SQL queries. 

For the last 10 years I had worked as developer using SQL all the time. I always feel happy when I have the opportunity to help more people to learn SQL or any other knowlege I could share.

# The Strategy Plan

1. Read the dataset from the "dataset" folder
2. Create a new Data Frame by grouping related data as it would be storage in the database tables
3. Create a sql file for the desired table in the folder "dumps"
4. For each row in the resulted Data Frame, record a equivalent INSERT syntax in the related sql file



In [1]:
import pandas as pd
dados = pd.read_excel("datasets/dados_modulo1.xlsx", sheet_name="vendas")
dados.head()

,ID da fila,ID da compra,Data da compra,Data de envio,Modo de envio,ID do cliente,Nome do cliente,Segmento,Cidade,Estado,...,Região,ID do produto,Categoria,Sub-categoria,Nome do produto,Moeda,Vendas,Quantidade,Desconto,Custo
0,1,MX-2014-143658,2019-10-02,2019-10-06,Classe padrão,SR-20575,Sofia Rocha,Varejo,Cidade do México,Distrito Federal,...,Norte,MAT-ET-10001014,Material de escritório,Etiquetas,"HON Etiquetas para organização de pastas, Ajus...",R$,130.8,3,0.0,45.6
1,2,MX-2012-155047,2017-10-15,2017-10-20,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,...,Sul,MÓV-MÓ-10004592,Móveis,Móveis,"Tenex Relógio de parede, Durável",R$,2521.6,8,0.0,907.2
2,3,MX-2012-155047,2017-10-15,2017-10-20,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,...,Sul,MÓV-ES-10000849,Móveis,Estantes,"Ikea Gaveteiro, Branco",R$,1932.8,2,0.0,540.8
3,4,MX-2012-155047,2017-10-15,2017-10-20,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,...,Sul,MAT-FI-10003284,Material de escritório,Fichários,"Cardinal Pasta argolada, Transparente",R$,354.4,4,0.0,49.6
4,5,MX-2012-155047,2017-10-15,2017-10-20,Classe padrão,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,...,Sul,MAT-AR-10002762,Material de escritório,Arte,"Sanford Tela, Tamanhos variados",R$,716.0,2,0.0,114.4


In [2]:
def scape(val) : 
    return str(val) if '@' == str(val)[0] else "'" + str(val).replace("'", "\\'") + "'"
    

In [25]:
def sqlInsert (table, cols, values) : 
    cols_str = "(" + ", ".join([str(col) for col in cols]) + ")"
    values_str = "(" + ", ".join([ scape(val) for val in values] ) + ")"

    filename = "dumps/" + table + ".sql"
    with open(filename, 'a+', encoding='utf-8') as f:
        f.write("\n INSERT IGNORE INTO " + table + " " + cols_str + ' VALUES ' + values_str + ";")
        

In [9]:
def sqlInsertFromDf(df, table_name, map_col_names ) : 
    
    feat_names = [feature_name for feature_name in map_col_names]
    col_names = [map_col_names[feature_name] for feature_name in map_col_names]
    
    for index, row in df.iterrows():
        values = [ row[feat_name] for feat_name in feat_names ] 
        sqlInsert(
            table_name, 
            col_names, 
            values 
        )
    

# Table and data description

- "produtos" is the table for product entity 
- "clientes" is the table for customer entity
- "compras" is a transactional table for purchases, where a customer from "clientes" has bought some quantity of product from "produtos"
- "envios" is a table with the shipping information for the purchases from "compras" table 
- "categorias" is the table of category entity
- "produto_categoria" is a relational table which links a product to categories

In [26]:

map_feature_col = {
    'ID da compra' : 'id', 
    'Data da compra' : 'data_compra', 
    'ID do cliente' : 'id_cliente'
}

index_name = list(map_feature_col)[0]

compras = dados.groupby(by=[index_name]).first().reset_index()[list(map_feature_col)].copy()
sqlInsertFromDf(compras, 'compras', map_feature_col)


In [33]:
map_feature_col = {
    'ID da compra' : 'id_compra', 
    'ID do produto' : 'id_produto', 
    'Quantidade' : 'quantidade',
    'Desconto': 'desconto',
    'Vendas': 'subtotal'
}

items_compra = dados.groupby(by=['ID da compra', 'ID do produto']).first().reset_index()[list(map_feature_col)].copy()
sqlInsertFromDf(items_compra, 'compra_items', map_feature_col)


In [28]:
map_feature_col = {
    'ID da compra' : 'id_compra', 
    'Modo de envio' : 'modo_envio', 
    'Data de envio' : 'data_envio', 
    'Cidade' : 'cidade', 
    'Estado' : 'estado', 
    'País' : 'pais', 
    'Região' : 'regiao' 
}

index_name = list(map_feature_col)[0]

envios = dados.groupby(by=[index_name]).first().reset_index()[list(map_feature_col)].copy()
sqlInsertFromDf(envios, 'envios', map_feature_col)


In [29]:
map_feature_col = {
    'ID do produto' : 'id', 
    'Nome do produto' : 'produto'
}

index_name = list(map_feature_col)[0]

produtos = dados.groupby(by=[index_name]).first().reset_index()[list(map_feature_col)].copy()
sqlInsertFromDf(produtos, 'produtos', map_feature_col)

In [30]:
map_feature_col = {
    'ID do cliente' : 'id', 
    'Nome do cliente' : 'nome'
}

index_name = list(map_feature_col)[0]

clientes = dados.groupby(by=[index_name]).first().reset_index()[list(map_feature_col)].copy()
sqlInsertFromDf(clientes, 'clientes', map_feature_col)

In [34]:
tabela_categoria = 'categorias'
categorias = dados.groupby(by=['Categoria', 'Sub-categoria']).first().reset_index()[['Categoria', 'Sub-categoria']]

for current_category in categorias['Categoria'].unique() : 
    subcategorias = dados.loc[dados['Categoria'] == current_category].groupby(by=['Sub-categoria']).first().copy()
    subcategorias = subcategorias.reset_index()['Sub-categoria'].tolist()
    sqlInsert(tabela_categoria, ['categoria'], [current_category])
    with open( 'dumps/' + tabela_categoria  + '.sql', 'a+', encoding='utf-8') as f:
        f.write("\n SET @last_category_id = LAST_INSERT_ID();")
    for subcat in subcategorias : 
       sqlInsert(tabela_categoria, ['categoria', 'parent_id'], [subcat, '@last_category_id'])    




In [37]:
prod_cat = dados.groupby(by=['ID do produto', 'Sub-categoria']).first().reset_index()[['ID do produto', 'Sub-categoria']]

for index, row in prod_cat.iterrows():
    
    prod_id = scape(row['ID do produto'])
    cat_id = scape(row['Sub-categoria'])

    with open('dumps/produto_categoria.sql', 'a+', encoding='utf-8') as f:
        f.write("\n INSERT IGNORE INTO produto_categoria (id_produto, id_categoria ) SELECT " + prod_id + ", categorias.id FROM categorias WHERE categorias.categoria LIKE " + cat_id + ";")
        
